# Projeto de algoritmos

#### Professor Mauricio Serrano

#### Davi Alves Bezerra - FGA UNB

#### Grafos1 - Sistema de recomendação baseado em grafos

---------------------------------------------------------------

Como visto, um grafo é capaz de representar conexões entre individuos, contas bancarias e até grupos mal intencionados. Há alguns datasets disponiveis pela universidade de [Stanford](https://snap.stanford.edu/data/) para analise de redes. Os dados foram coletados por meio de crawling do site da Amazon e contêm metadados de produtos e informações de revisão sobre 548.552 produtos diferentes (Livros, CDs de música, DVDs e fitas de vídeo VHS). Nesse projeto irei usar somente os livros a principio para diminuir a quantidade de dados. Há o total de 393561 livros, de gêneros diversos. Os dados usados são de 2006.

### Disposição dos dados:

- Id: Id do produto (numero entre 0, ..., 548551)
- ASIN: Amazon Standard Identification Number (Número de identificação padrão da Amazon)
- title: Nome ou titulo do produto
- group: Grupo do produto, ou tipo (Livro, DVD, Video or Musica)
- salesrank: Rank de vendas da Amazon
- similar: ASINs de produtos comprados em conjunto (as pessoas que compram X também compram Y)
- categories: Local na hierarquia de categorias do produto ao qual o produto pertence (separado por |, ID da categoria)
- reviews: Informações da avaliação do produto: tempo, ID do usuário, classificação, número total de votos na avaliação, número total de votos de utilidade (quantas pessoas acharam a avaliação útil)

In [ ]:
# Download
!wget https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz

In [5]:
# unzip dataset
!gzip -dk amazon-meta.txt.gz

Agora vamos identificar o que poderia ser nossos nós e vertices para montar o grafo.
Inicialmente vamos criar uma rede de relacionamento usando apenas algumas compras para ter uma ideia.

In [6]:
from pyvis.network import Network
import pandas as pd
import re

In [14]:
# função para selecionar uma quantia dentro do dataset
def get_sample_dataset(raw_data, len_sample, data_name="amazon-meta"):
    count = 1
    left_text = 0
    dataset = []
    
    if data_name=="amazon-meta":
        amazon_data = raw_data[3:]
        
        for i, line in enumerate(amazon_data):
            if line == '\n':
                dataset.append(amazon_data[left_text:i])
                left_text = i
                
                if count==len_sample:
                    return dataset
                count+=1

# função para separar itens desejados
def get_amazon_metadata(dataset):
    
    discontinued_product = re.findall(r"discontinued product", dataset)
    id_product = re.findall(r"Id:.*?\n", dataset)
    asin_product = re.findall(r"ASIN:.*?\n", dataset)
    cutomer_product = re.findall(r"cutomer:.*?\w+", dataset)
    
    if not asin_product or discontinued_product or not cutomer_product:
        return None
    
    rate_cutomer = re.findall(r"rating:\s(\d+)", dataset)
    similar_asin_products = re.findall(r"similar:.*?\n", dataset)
    category_product = re.findall(r"group:.*?\n", dataset)
    title_product = re.findall(r"title:(.*?)\n", dataset)

    ids = list(filter(None, id_product[0].split(" ")))[1].replace('\n', '')
    source = [i.split(' ')[-1] for i in cutomer_product]
    target = list(len(source) * (asin_product[0].split(" ")[1].replace('\n', ''),))
    weight = rate_cutomer[1:]
    name = title_product[0].strip()

    return ids, source, target, weight, name

# função para gerar o grafo de forma visual
def create_visual_network(customer, book, stars):
    got_net = Network(notebook=True)

    got_net.barnes_hut()

    edge_data = zip(customer, book, stars)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        got_net.add_node(src, src, title=src)
        got_net.add_node(dst, dst, title=dst)
        got_net.add_edge(src, dst, value=w)

    neighbor_map = got_net.get_adj_list()

    for node in got_net.nodes:
                    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                    node["value"] = len(neighbor_map[node["id"]])

    return got_net.show("books.html")

In [8]:
# read amazon-meta
with open('amazon-meta.txt', 'r', encoding="utf8", errors='ignore') as f:
    lines = f.readlines()

Em `num_samples` podemos escolher quantas compras vamos usar para visualizar a rede. Nesse exemplo usaremos 100 amostras.

In [9]:
num_samples = 100
dataset = get_sample_dataset(lines, num_samples)
dataset = ["".join(item) for item in dataset]

In [10]:
# filtro para remover os produtos que retornaram None, alguns deles foram descontinuados
products_filtered = list(filter(None, [get_amazon_metadata(i) for i in dataset]))

In [12]:
# junção das amostras para visualização
customer = list()
book = list()
stars = list()

for i in products_filtered:
    customer += i[1]
    book += i[2]
    stars += i[3]

In [15]:
create_visual_network(customer, book, stars)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
